## Execution des libraries

In [ ]:
import sys
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy 
import matplotlib.ticker as ticker
import cs
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Le fonction nextRandomValue
#### il Produit le premier seuil random  
#### le input de ce fonction est peviousThreshold qui prends une valeur par hasard entre 0 et 90 de multiple 5
#### on définit des limits pour les suivantes seuils random avec le min 0 et le max 90 de multiple 5
#### le output de ce fonctionest nextValue qui donne suivante seuil random par rapport les limites et premier seuil

In [ ]:
def nextRandomValue(previousThreshold): 
    diffmax=25
    lowerLimit=max(0, previousThreshold - diffmax)
    upperLimit=min(95, previousThreshold + diffmax)
    nextValue = random.randrange(lowerLimit, upperLimit, 5) 
    return nextValue

## Création un nouvel audiogram
#### il définit premier audiogramme random avec les limites de seuil (nextRandomValue) et les frequences,
#### le input de ce fonction est VF le vectuer de frequence qu'on va le donner apres dans le command line
#### audiogramme au début est une vecteur vide qu'on va le remplisse avec des valeurs randoms de seuil pour produire le premier audiogramme, avec "append" on joint tous les seuils randoms produit pour obtenir l'audigrame,
#### on trouve les seuils randoms avec le fonction de "nextrandomvalue" pour toutes les frequences "VF",
#### le output de ce fonction est audiogram qui donne le premier audigramme avec ses parametres de seuil et frequence   

In [ ]:
def nouvelAudiogram(VF):
    vectorFrequency=VF
    previous = random.randrange(0, 95, 5)
    audiogram = [] 
    audiogram.append(previous) 
    for cpt in range(len(VF)-1):
        nextValue = nextRandomValue(previous)
        audiogram.append(nextValue)
        previous = nextValue
    return(audiogram)

## Le fonction differenceEntreDeuxAudiogrammes
#### il définit la difference entre le premier audiogramme random et le deuixieme
#### le input de ce fonction est les 2 audiogrammes 
#### score diff est le minimum difference entre 2  audiogrammes qui peut etre 0
#### On calcule un score de différence basé sur la corrélation
#### on transforme les coefficients de correlations (1 0 -1) à (0 50 100)
#### score diff prends les valeurs de diagnal de matrice coef correlations transformés 
#### le output de ce fonction est la valeur de correlations entre 2 audigrammes 

In [ ]:
def differenceEntreDeuxAudiogrammes(audio1, audio2):
    scoreDif = 0 
    matrice = ((numpy.corrcoef(audio1, audio2)*(-1)) + 1) * 50 
    scoreDif = matrice[0,1] 
    return(scoreDif)

## Le fonction creerAudiogrammeDifferent
#### il crée un nouvel audiogramme très différent des audiogrammes présents dans dfAudiogrammes
#### le input de ce fonction est le dfAudiogrammes,nombre audiogramme qu'on a besoin et le vecteur de frequence
#### Pour cela, il génère nbAudiogrammesRandom audiogrammes random avec differents frequences et retourne l'audiogramme le plus différent de tous les audiogrammes présents dans dfAudiogrammes 
#### Créer une dataframe temporaire pour mettre les audiogrammes random
#### dans le tous premier boucle on crée nbAudiogrammesRandom audiogrammes random et les insère dans la df temporaire et il trouve nbRandomAudiograms audiogrammes random
#### comme que il crée un audiogramme random avec Using DataFrame.insert() pour ajouter un colonne après pour chacun des audiogrammes random
#### dans le deuxième boucle avec l'input dfAudiogrammeTemp il calcul la différence moyenne avec tous les audiogrammes présents dans dfAudiogrammes
#### On calcule les differences entre chaque audiogramme random et chaque audiogramme de dfFinale
#### avec dfDifferences['mean'] on ajoute les moyennes
#### [audiogramme =] il trouve l'audiogramme random le plus différent de tous les audiogrammes présents dans dfAudiogrammes
#### le output de ce fonction est pour retourner cet audiogramme

In [ ]:
def creerAudiogrammeDifferent (dfAudiogrammes, nbAudiogrammesRandomToUse, VF):
    dfAudiogrammesTemp=pd.DataFrame() 
    for i in range(nbAudiogrammesRandomToUse):
        audioRandom=nouvelAudiogram(VF)
        dfAudiogrammesTemp.insert(i, str(i), audioRandom, True) 
    dfDifferences  = pd.DataFrame()
    for i in list (dfAudiogrammesTemp):
        for j in list (dfAudiogrammes):
            difference = differenceEntreDeuxAudiogrammes(dfAudiogrammesTemp.loc[:,i], dfAudiogrammes.loc[:,j])
            dfDifferences.loc[i,j] = difference         
    dfDifferences['mean'] = dfDifferences.mean(axis=1)
    audiogramme = dfAudiogrammesTemp[dfDifferences['mean'].idxmax()]
    return audiogramme

## Boucle generale create_audiograms
#### on cree une autre fonction pour definir les audiograms final et les tracer
#### nbRandomAudiogramsWeWant On définit le nombre d'audiogrammes très différents qu'on veut
#### nbRandomAudiogramsToUse On définit le nombre d'audiogrammes random à utiliser pour trouver chaque audiogramme très différent
#### On crée la base dfFinale avec un premier audiogramme random
#### avec le boucle for il joint les audiogrammes tres differents l'un et l'autre et les mettre dans le dfFinale l'un apres l'autre

In [ ]:
def create_audiograms(nbrAudiogramsWanted,nbrAudiogramsbeUsed,VF):
    nbRandomAudiogramsWeWant = nbrAudiogramsWanted
    nbRandomAudiogramsToUse = nbrAudiogramsbeUsed
    dfFinale=pd.DataFrame(nouvelAudiogram(VF))
    for i in range (nbRandomAudiogramsWeWant):
        audiogrammeNouveau = creerAudiogrammeDifferent(dfFinale,nbRandomAudiogramsToUse, VF)
        dfFinale.append(audiogrammeNouveau)
        dfFinale.insert(i + 1, str(i + 1), audiogrammeNouveau, True)
        print("audiogramme " + str(i+1))
        dfFin=dfFinale.T
    dfFinale_save=dfFin.to_csv('df5000.csv', mode='a',header=False) 

## Le fonction main sert de point d'exécution pour tout programme
#### sys.argv :La liste des arguments de ligne de commande passés à un script Python
#### si on donne moin de 3 arguments il affiche rien mais il donne instruction de comment ecrire en terminal
#### 0=nom de fichier 1=1er parametere 2=2eme parametere 3=3eme parameter 
#### num1 est le nombre d'audiogrammes très différents qu'on veut
#### num2 est le nombre d'audiogrammes random à utiliser pour trouver chaque audiogramme très différent
#### vec_frequences est le vecteur frequence,changer le delimiter de vecteur frequence par virgule 
#### le boucle for genere 100 fois les audiogrammes tres differents avec plusieurs valeurs de frequences en terminal
#### et le condition if contrôle l'exécution du code

In [ ]:
def main():
    if len(sys.argv)<=3:
        print("you have to write: %run Audiograms.py x y; x:your first value for number of audiorams wanted and  y:your second value for the number of audiograms have to be used")
    else:
        num1=int(sys.argv[1]) 
        num2=int(sys.argv[2]) 
        vec_frequences = sys.argv[3].split(",") 
        for i in range(100):
            create_audiograms(num1,num2,vec_frequences)           
if (__name__=="__main__") : 
    main()

### "tous ces codes sont enregistré dans le un fichier Audiograms.py et on a juste besoin de tourner le code ci-dessous sur un fichiers comme celui-ci"
#### on a essayé de génerer 5000 audiogrammes mais parce que le temps de traitement est très long, on a préféré de répéter 100 fois 50 audiogrammes très différents

In [ ]:
%run Audiograms.py 50 50 125,250,500,750,1000,1500,2000,3000,4000,6000,8000,10000

## Nettoyage et interprétation des données

In [ ]:
df1 = pd.read_csv('Results.csv' , sep=';' , header=None)
df1.columns = df1.iloc[0]
df1=pd.DataFrame(df1)
df1=df1.drop(df1.index[0])
df1[["seuil_125","seuil_250","seuil_500","seuil_750","seuil_1000","seuil_1500","seuil_2000",
    "seuil_3000","seuil_4000","seuil_6000","seuil_8000","seuil_10000",
    "phoneme_correct"]] = df1[["seuil_125","seuil_250","seuil_500","seuil_750","seuil_1000","seuil_1500","seuil_2000",
    "seuil_3000","seuil_4000","seuil_6000","seuil_8000","seuil_10000",
    "phoneme_correct"]].apply(pd.to_numeric)
# check records
df2["erreurs_phono"] = df2.erreurs_phono.str.replace(']','')
df2["erreurs_phono"] = df2.erreurs_phono.str.replace('[','')
df2=(df2.set_index(df2.columns.drop('erreurs_phono',1).tolist()).erreurs_phono.str.split('#', expand=True).stack().reset_index().rename(columns={0:'erreurs_phono'}).loc[:, df2.columns])
df2['mot'] =df2[['phoneme_cible', 'phoneme_reconnu']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df2 = df2.rename({'seuil_125': '125', 'seuil_250': '250','seuil_500': '500','seuil_750': '750','seuil_1000': '1000','seuil_1500': '1500','seuil_2000': '2000','seuil_3000': '3000'
                    ,'seuil_4000': '4000','seuil_6000': '6000','seuil_8000': '8000','seuil_10000': '10000'}, axis=1)
df2

## Calculer le pourcentage d'erreur 

In [ ]:
pourcentage=pd.DataFrame(100*df1.groupby(['id_audiogramme', 'phoneme_correct']).size() / df1.groupby(['id_audiogramme']).size())
pourcentage.to_csv('pourcentage.csv') 
pourcentage=pd.read_csv('pourcentage.csv' , sep=',' , header=None)
pourcentage.columns = pourcentage.iloc[0]
pourcentage=pd.DataFrame(pourcentage)
pourcentage=pourcentage.drop(pourcentage.index[0])
pourcentage.columns=['id_audiogramme', 'phoneme_correct','pourcentage']
df_pourcentage=pourcentage.loc[(pourcentage['pourcentage'] >50)& (pourcentage['phoneme_correct'] == '1') ]
df_pourcentage

In [ ]:
df2=df1[df1.set_index(['id_audiogramme']).index.isin(df_pourcentage.set_index(['id_audiogramme']).index)]
df2['mot'] =df2[['phoneme_cible', 'phoneme_reconnu']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df2 

In [ ]:
df_na=df2[df2.isna().any(axis=1)]
df_na = df_na.rename({'seuil_125': '125', 'seuil_250': '250','seuil_500': '500','seuil_750': '750','seuil_1000': '1000','seuil_1500': '1500','seuil_2000': '2000','seuil_3000': '3000'
                    ,'seuil_4000': '4000','seuil_6000': '6000','seuil_8000': '8000','seuil_10000': '10000'}, axis=1)
df_na

In [ ]:
df2["erreurs_phono"] = df2.erreurs_phono.str.replace(']','')
df2["erreurs_phono"] = df2.erreurs_phono.str.replace('[','')
df2=(df2.set_index(df2.columns.drop('erreurs_phono',1).tolist()).erreurs_phono.str.split('#', expand=True).stack().reset_index().rename(columns={0:'erreurs_phono'}).loc[:, df2.columns])
df2['mot'] =df2[['phoneme_cible', 'phoneme_reconnu']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df2 = df2.rename({'seuil_125': '125', 'seuil_250': '250','seuil_500': '500','seuil_750': '750','seuil_1000': '1000','seuil_1500': '1500','seuil_2000': '2000','seuil_3000': '3000'
                    ,'seuil_4000': '4000','seuil_6000': '6000','seuil_8000': '8000','seuil_10000': '10000'}, axis=1)
df2

In [ ]:
df_count=pd.DataFrame(df2.mot.value_counts().head(48))
df_count.to_csv('df_count.csv') 
df_count=pd.read_csv('df_count.csv' , sep=',' , header=None)
df_count.columns = df_count.iloc[0]
df_count=pd.DataFrame(df_count)
df_count=df_count.drop(df_count.index[0])
df_count.columns=['mot', 'nbr']
mot_list = df_count["mot"].tolist()
print(mot_list)

In [ ]:
motList = ['tR', 'Rs', 'dg', 'kp', 'fR', 'tp', 'vk', 'dt', 'vR', 'vp', 'nb', 'bp', 'mb', 'Rf', 'fk', 'vf', 'fp', 'nm', 'mn', 'ml', 'sf', 'bv', 'bf', 'vt', 'RS', 'sp', 'df', 'zg', 'dk', 'vg', 'vZ', 'dp', 'kt', 'mz', 'bg', 'sR', 'bR', 'Sf', 'bz', 'ng', 'dz', 'zs', 'dR', 'db', 'vz', 'fs', 'mv', 'pR', 'bd', 'zR', 'ln', 'tl', 'lm', 'nd', 'zZ', 'gk', 'dv', 'Ss', 'sk']
df_filtered = df2[df2['mot'].isin(motList)]
df_filtered

In [ ]:
df_new=df_na[df_na.set_index(['phoneme_cible']).index.isin(df_filtered.set_index(['phoneme_cible']).index)]
df_new

In [ ]:
df_append=df_filtered.append(df_new)
df_append

In [ ]:
df_append=df_filtered.append(df_new)
df_append[["id_audiogramme"]] = df_append[["id_audiogramme"]].apply(pd.to_numeric)
sorted_df = df_append.sort_values(by=['id_audiogramme'], ascending=True)
sorted_df=sorted_df[['id_audiogramme', '125', '250', '500', '750',
       '1000', '1500', '2000', '3000', '4000',
       '6000', '8000', '10000', 'mot']]
sorted_df

In [ ]:
df_filtered.fillna('', inplace=True)
df_new1=(df_filtered.erreurs_phono).str.get_dummies()
result1 = pd.concat([df_filtered, df_new1], axis=1, ignore_index=False)
result1

In [ ]:
result1=result1[['id_audiogramme', '125', '250', '500', '750', '1000', '1500', '2000',
       '3000', '4000', '6000', '8000', '10000',
       '+anterieur', '+arriere', '+continu', '+coronal', '+haut', '+nasal',
       '+sonant', '+voise', '-anterieur', '-arriere', '-continu', '-coronal',
       '-haut', '-nasal', '-sonant', '-voise']]
result1.columns

In [ ]:
df_trait=result1.melt(id_vars=['id_audiogramme', '125', '250', '500', '750', '1000', '1500', '2000',
       '3000', '4000', '6000', '8000', '10000'], 
        var_name="erreur", 
        value_name="binaire")
df_trait

In [ ]:
#Loading the .txt file
df = pd.read_csv('results_df_mean.txt' , sep='/' , header=None)
df.columns=['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11']
df[['X12','X13']]=df.X11.str.split("[",expand=True,)
df[['X14','X15']]=df.X13.str.split("]",expand=True,)
df[['X16','X17']]=df.X15.str.split(",",expand=True,)
df

In [ ]:
df.loc[df['X14'] =='ch', 'X14'] = "S"
df.loc[df['X17'] =='ch', 'X17'] = "S"
df.loc[df['X14'] =='j', 'X14'] = "Z"
df.loc[df['X17'] =='j', 'X17'] = "Z"
df.loc[df['X14'] =='r', 'X14'] = "R"
df.loc[df['X17'] =='r', 'X17'] = "R"
df

In [ ]:
df['erreur'] =df[['X14', 'X17']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df=df[['X10','erreur']]
df

In [ ]:
df.columns=['x','erreur']
df['x']=df['x'].str.replace(r'\(', '')
df['x']=df['x'].str.replace(r'\)', '')
df

In [ ]:
df_res= pd.read_csv('df_mean.csv' , sep=',' , header=None)
df_res.columns = df_res.iloc[0]
df_res=pd.DataFrame(df_res)
df_res=df_res.drop(df_res.index[0])
df_res.columns=['X125','X250','X500','X750','X1000','X1500','X2000','X3000','X4000','X6000','X8000','X10000']
df_res

In [ ]:
df_res = df_res.assign(erreur = [ "RS", "Rf", "Rs", "SS", "Sf", "Ss" ,"bR", "bb" ,"bd", "bf", "bg" ,"bp" ,"bv","bz", "dR" ,"db" ,"dd" ,"df" ,"dg", "dk","dp" ,"dt" ,"dv" ,"dz" ,"fR" ,"ff", "fk", "fp" ,"fs" ,"gg" ,"gk" ,"kk" ,"kp" ,"kt" ,"ll" ,"lm", "ln", "mb","ml" ,"mm" ,"mn" ,"mv" ,"mz" ,"nb" ,"nd" ,"ng", "nm" ,"nn" ,"pR", "pp" ,"sR" ,"sf" ,"sk" ,"sp", "ss" ,"tR" ,"tl" ,"tp" ,"tt" ,"vR" ,"vZ" ,"vf" ,"vg", "vk" ,"vp" ,"vt" ,"vv" ,"vz" ,"zR" ,"zZ" ,"zg" ,"zs" ,"zz"])
df_res

In [ ]:
pos = [8,15,47,34,29,3,7,39,49,31,25,54,16,66,58,72]
df_res.drop(df_res.index[pos], inplace=True)
df_res

In [ ]:
df_res=df_res.reset_index(drop=True)
df_res

In [ ]:
cols = ['X125', 'X250', 'X500',"X750","X1000","X1500","X2000","X3000","X4000","X6000","X8000","X10000"]
df_res['x'] = df_res[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df_res